Model Evaluation and Hyperparameter Tuning

In [18]:
import pandas as pd
import numpy as np


# Load dataset

In [20]:
from sklearn.datasets import load_iris

data = load_iris()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = pd.Series(data.target)

In [21]:
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler

# Split the dataset and Standardize the features

In [23]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [24]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

# Define models

In [26]:

models = {
    'LogisticRegression': LogisticRegression(max_iter=200),
    'RandomForest': RandomForestClassifier(),
    'SVM': SVC()
}

In [27]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report


# Train and evaluate models

In [28]:

results = []

print("Initial Evaluation of Models:\n")

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    results.append({
        'Model': name,
        'Accuracy': accuracy_score(y_test, y_pred),
        'Precision': precision_score(y_test, y_pred, average='weighted'),
        'Recall': recall_score(y_test, y_pred, average='weighted'),
        'F1 Score': f1_score(y_test, y_pred, average='weighted')
    })
    
    print(f"Model: {name}")
    print(classification_report(y_test, y_pred))
    print("-" * 50)

results_df = pd.DataFrame(results)
print("\nSummary of Initial Results:")
print(results_df)

Initial Evaluation of Models:

Model: LogisticRegression
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      1.00      1.00         9
           2       1.00      1.00      1.00        11

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30

--------------------------------------------------
Model: RandomForest
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      1.00      1.00         9
           2       1.00      1.00      1.00        11

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30

--------------------------------------------------
Model: SVM
              precision    recall  f1-score   su

# Hyperparameter tuning for Random Forest (GridSearchCV)

In [30]:

param_grid_rf = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10]
}


In [12]:
grid_search_rf = GridSearchCV(RandomForestClassifier(), param_grid_rf, cv=5, scoring='f1_weighted', n_jobs=-1)
grid_search_rf.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'max_depth': [None, 10, 20],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [50, 100, 200]},
             scoring='f1_weighted')

# Hyperparameter tuning for SVM (RandomizedSearchCV)

In [31]:

param_dist_svm = {
    'C': np.logspace(-3, 2, 6),
    'gamma': ['scale', 'auto'],
    'kernel': ['linear', 'rbf', 'poly']
}

random_search_svm = RandomizedSearchCV(SVC(), param_distributions=param_dist_svm, n_iter=10, cv=5, scoring='f1_weighted', n_jobs=-1, random_state=42)
random_search_svm.fit(X_train, y_train)


RandomizedSearchCV(cv=5, estimator=SVC(), n_jobs=-1,
                   param_distributions={'C': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02]),
                                        'gamma': ['scale', 'auto'],
                                        'kernel': ['linear', 'rbf', 'poly']},
                   random_state=42, scoring='f1_weighted')

# Final evaluation of best models

In [32]:

final_models = {
    'RandomForest_Optimized': grid_search_rf.best_estimator_,
    'SVM_Optimized': random_search_svm.best_estimator_
}

In [15]:
print("\nAfter Hyperparameter Tuning:\n")

final_results = []

for name, model in final_models.items():
    y_pred = model.predict(X_test)
    
    final_results.append({
        'Model': name,
        'Accuracy': accuracy_score(y_test, y_pred),
        'Precision': precision_score(y_test, y_pred, average='weighted'),
        'Recall': recall_score(y_test, y_pred, average='weighted'),
        'F1 Score': f1_score(y_test, y_pred, average='weighted')
    })
    
    print(f"Optimized Model: {name}")
    print(classification_report(y_test, y_pred))
    print("-" * 50)

final_results_df = pd.DataFrame(final_results)
print("\nSummary of Tuned Model Results:")
print(final_results_df)


After Hyperparameter Tuning:

Optimized Model: RandomForest_Optimized
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      1.00      1.00         9
           2       1.00      1.00      1.00        11

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30

--------------------------------------------------
Optimized Model: SVM_Optimized
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      1.00      1.00         9
           2       1.00      1.00      1.00        11

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30

--------------------------------------------------

Summary of Tuned Model Results:
 

# Best model selection

In [33]:


best_model = final_results_df.sort_values(by='F1 Score', ascending=False).iloc[0]
print(f"\n✅ Best Performing Model: {best_model['Model']} with F1 Score: {best_model['F1 Score']:.4f}")


✅ Best Performing Model: RandomForest_Optimized with F1 Score: 1.0000
